In [ ]:
import os
import pandas as pd
from lxml import html
import requests
import re
import sys
from datetime import datetime

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import time


# Input the CIK list for the fund family you are looking at

In [ ]:
fund_name = 'blackrock'


CIK_LIST = ['844779', '1398078', '774013', '1062806', '1026144', '1062805', '276463', '1097077', '1137391', '1160864', '1181249', '1159038', '353281', '834237', '97098', '216557', '765199', '814507', '790525', '110055', '1159040', '1137393', '1176194', '877151', '1169029', '1167467', '319108', '1051003', '1137440', '1167470','230382','1097293']




In [ ]:
cur_wd = os.getcwd()
if 'trunk' in cur_wd:
    output_directory = os.path.join("\\".join(cur_wd.split('\\')[0:-1]),'output')
else:
    output_directory = os.path.join("\\".join(cur_wd.split('\\')[0:-1]),'output','working')
    
cur_wd = os.path.join("\\".join(cur_wd.split('\\')[0:-1]),'data')
print('data directory')
print(cur_wd)
print('output directory')
print(output_directory)


# Helper Functions for Analysis

In [ ]:
def get_cleaned(lines):
    cleaned = []
    for i in lines:
        #gets rid of tags
        cleaned_i = re.sub('<[^>]*>', ' ', i)
        #gets rid of &nbsp; and replaces with space
        cleaned_i = re.sub('&nbsp;', ' ', cleaned_i)
        #re.sub("<.*?>","",st)
        if not re.match(r'^\s*$', cleaned_i):
            cleaned.append(cleaned_i)
    return cleaned

def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]


def hasNumbers(inputString):
    return bool(re.search(r'\d', inputString))


def remove_junk(addline):
    addline = re.sub('<[^>]*>', ' ', addline)
    addline = re.sub('&nbsp;', ' ', addline)
    addline = re.sub('&#174;', ' ', addline)
    addline = re.sub('&#8480;', ' ', addline)
    addline = re.sub(r'&reg;',' ',addline)
    addline = re.sub(r'\n', ' ', addline)
    addline = re.sub(r'\t', ' ', addline)
    return addline


In [ ]:
weblink = {}
for CIK in CIK_LIST:
    weblink[CIK]  = []
    txt_files = os.listdir(os.path.join(cur_wd, CIK, 'n-q'))
    for i in txt_files:
        weblink[CIK].append(r"https://www.sec.gov/Archives/edgar/data/" + str(CIK)+r"/"  + i)

In [ ]:
weblink

get panel 2

In [ ]:
panel2 = pd.DataFrame()
error_panel2 = []

    
for CIK in CIK_LIST:
    txt_files = os.listdir(os.path.join(cur_wd, CIK, 'n-q'))
    
    for i in txt_files:
        if '.csv' not in i:
            try:
                series = {}
                series_keys = ['SERIES-ID', 'OWNER-CIK', 'SERIES-NAME']
                for s in series_keys:
                    series[s] = []

                contract = {}
                contract_keys = ['CLASS-CONTRACT-ID','CLASS-CONTRACT-NAME','CLASS-CONTRACT-TICKER-SYMBOL', 'LAST-READ-SERIES']
                for c in contract_keys:
                    contract[c] = []

                uncleaned_file = open(os.path.join(cur_wd, CIK, 'n-q',i), mode='r') 
                for u in uncleaned_file:
                    

                    if 'COMPANY CONFORMED NAME' in u:
                        conformed_name = u.replace('COMPANY CONFORMED NAME:', '').strip()

                    if 'FILED AS OF DATE:' in u:
                        filed_date = str(u).replace('FILED AS OF DATE:','').strip()

                    #series information
                    for s in series_keys:
                        if s in u:
                                series[s].append(str(u).replace('<' + s + '>', "").strip())
                    for c in contract_keys:
                        if c != 'LAST-READ-SERIES':
                            if c in u:
                                contract[c].append(str(u).replace('<' + c + '>', "").strip())
                                if c == 'CLASS-CONTRACT-ID':
                                    contract['LAST-READ-SERIES'].append(series['SERIES-ID'][-1])


                series = pd.DataFrame.from_dict(series, orient = 'index').T
                contract = pd.DataFrame.from_dict(contract, orient = 'index').T


                for s in series_keys:
                    contract.loc[:, s] = ''

                for ix,row in contract.iterrows():
                    for s in series_keys:
                        contract.loc[contract.index == ix, s] = series.loc[series['SERIES-ID'] == row['LAST-READ-SERIES'],s].values[0]
                contract.loc[:,'file_read'] = i
                contract.loc[:,'date_filed'] = filed_date
                contract.loc[:, 'company conformed name'] = conformed_name
                contract = contract.drop(['LAST-READ-SERIES'], axis=1)

                if panel2.empty:
                    panel2 = contract.copy()
                else:
                    panel2 = pd.concat([panel2, contract], axis = 0)
            except ValueError:
                url = r'https://www.sec.gov/Archives/edgar/data/' + CIK + r'/' + i
                error_panel2.append(url)
                

In [ ]:
print(panel2)
panel2.to_csv(os.path.join(output_directory, fund_name + '_panel2.csv'), sep = ',')

## Test on One FILE

In [ ]:

CIK  ='774013'
text =  '0001171200-09-000205.txt'


In [ ]:
url = r'https://www.sec.gov/Archives/edgar/data/' + CIK + r'/' + text.replace('.txt', '').replace('-','')  + r'/' + text
print(url)
tables = pd.read_html(url)
print(len(tables))


In [ ]:
"""

from bs4 import BeautifulSoup
import requests

url=r'https://www.sec.gov/Archives/edgar/data/' + CIK +  r'/' + text

# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

# Parse the html content
soup = BeautifulSoup(html_content, "lxml")
#print(soup.prettify()) # print the parsed data of html

table_tags = soup.find_all("table")
table = []

for i in table_tags:
    print(table_tags)

"""


In [ ]:
#read a table

In [ ]:
df = tables[0].dropna(axis = 0, how = 'all')
df = df.dropna(axis = 0, subset = [df.columns[0]])
cleaned_df = pd.DataFrame()
df = df.dropna(thresh = 3, axis = 0)
headername = ''
for count, row in df.iterrows():
    name = row[0]
    if ':' in name:
        headername = name
    if name.split(' ')[0] == 'Series' or name[0].isdigit():
        name = headername + ' ' + name
        
    if len(row.dropna().values) < 3:
        print(row)
        
    add = pd.DataFrame(row[1:].dropna())
    add = add[~add[add.columns[0]].str.contains('\D+')]
    if len(add) == 2:
        add = add.transpose().dropna()
        add.columns = ['holdings shares', 'holdings value']
        add['holdings name'] = name
        
        cleaned_df = pd.concat([cleaned_df, add], axis = 0, sort = False)
        
cleaned_df.index = pd.RangeIndex(len(cleaned_df.index))
print(cleaned_df)

In [ ]:
count = 0
for df in tables:
    df = df.dropna(axis = 0, how = 'all')
    df = df.dropna(axis = 1, how = 'all')
    
    if not df.empty:
        print('count')
        print(count)
        print(' ')
        print(df)
    count = count + 1

In [ ]:
def get_report_fund(df):
    reportdate = ''
    fundname = ''
    
    col_count = 0
    for col in df.columns:
        check_name = df[col][df[col].apply(lambda x: re.sub(r'\s', '', str(x)).lower()).str.contains('scheduleofinvestments')]
        if len(check_name) >= 1:
            check_name_ix = check_name.index[0]
            if 'continued' not in str(df.loc[df.index == check_name_ix, col].values[0]) and 'audited' in str(df.loc[df.index == check_name_ix, col].values[0]):
                reportdate = str(df.loc[df.index == check_name_ix, col].values[0])
                reportdate = re.sub(r'\s', '',reportdate.encode('ascii','ignore').decode().lower()).replace(r'(unaudited)',"").replace(r'schedule',"").replace(r'of',"").replace(r'investments',"")

                fundname = str(df.loc[df.index == check_name_ix, df.columns[col_count + 1]].values[0])
                fundname = re.sub(r'\s', '',fundname.encode('ascii','ignore').decode().lower()).replace('(percentagesshownarebasedonnetassets)', '')

        col_count = col_count + 1
    return reportdate, fundname

In [ ]:
def check_level_columns(df):
    check_found = False
    for col in df.columns:
        check_name = df[col][df[col].apply(lambda x: re.sub(r'\s', '', str(x)).lower()).str.contains('level1')]
        if len(check_name) > 0:
            check_found = True
    return check_found

In [ ]:
def get_holdings(df, reportdate, fundname, headername):
    
    cleaned_df = pd.DataFrame()

    check_level = check_level_columns(df)

    if len(df) > 2 and len(df.columns) >= 3 and not check_level:

        for count, row in df.iterrows():
            name = str(row[0])
            if ':' in name:
                headername = name
            if name.split(' ')[0] == 'Series' or name[0].isdigit():
                name = headername + ' ' + name

            add = pd.DataFrame(row[1:].dropna())
            add = add[~add[add.columns[0]].str.contains('\D+', na=False)]

            if name[-1] != r'%' and name != 'adr' and name != 'class' and name != 'securities' \
            and name != 'total' and name != 'affiliate':

                if len(add) == 2:
                    add = add.transpose().dropna()
                    add.columns = ['holdings shares', 'holdings value']
                    add['holdings name'] = name
                    add.loc[:,'reporting_date'] = reportdate
                    add.loc[:,'fund name'] = fundname
                    add.loc[:, 'identifier'] = ''

                    cleaned_df = pd.concat([cleaned_df, add], axis = 0, sort = False)
                    cleaned_df.index = pd.RangeIndex(len(cleaned_df.index))
                    
        
    return cleaned_df, headername
    

In [ ]:
def get_keys(holdings):
    for count, row in holdings.iterrows():
        keys = re.findall(r'\([^()]*\)', row['holdings name'])
        keylist = ''
        keycheck = []
        for check in keys:
            if len(check)<= 3:
                keycheck.append(check)
        if len(keys) > 0:
            for k in keys:
                if 'Acquired' not in k:
                    if len(k) <= 3:
                        keylist = keylist + k
                        holdings.loc[count, 'holdings name'] = holdings.loc[count, 'holdings name'].replace(k,'')

                else:
                    holdings.loc[count, 'holdings name'] = holdings.loc[count, 'holdings name'].replace(k,'')
                    holdings.loc[count, 'acq name'] = row['holdings name'].split(',')[0]
                    acq_data = k.split(' ')
                    holdings.loc[count, 'acq date'] = acq_data[1].replace(',','')
                    holdings.loc[count, 'acq cost'] = ''.join(re.findall("\d+", acq_data[-1]))

        holdings.loc[count, 'key'] = ",".join(keycheck)
    return holdings
    

In [ ]:
    df = tables[0]
    reportdate = ''
    fundname = ''
    
    col_count = 0
    for col in df.columns:
        check_name = df[col][df[col].apply(lambda x: re.sub(r'\s', '', str(x)).lower()).str.contains('scheduleofinvestments')]
        if len(check_name) >= 1:
            check_name_ix = check_name.index[0]
            if 'continued' not in str(df.loc[df.index == check_name_ix, col].values[0]) and 'audited' in str(df.loc[df.index == check_name_ix, col].values[0]):
                reportdate = str(df.loc[df.index == check_name_ix, col].values[0])
                reportdate = re.sub(r'\s', '',reportdate.encode('ascii','ignore').decode().lower()).replace(r'(unaudited)',"").replace(r'schedule',"").replace(r'of',"").replace(r'investments',"")

                fundname = str(df.loc[df.index == check_name_ix, df.columns[col_count + 1]].values[0])
                fundname = re.sub(r'\s', '',fundname.encode('ascii','ignore').decode().lower()).replace('(percentagesshownarebasedonnetassets)', '')
                print(fundname)

In [ ]:
for df in tables:
    #remove all rows and columns of nas
    df = df.dropna(axis = 0, how = 'all')
    df = df.dropna(axis = 1, how = 'all')
    
    if not df.empty:
        try:
            reportdate, fundname = get_report_fund(df)
            if reportdate != '':
                print(reportdate)
                print(fundname)
                found = True
        except :
            pass

In [ ]:


holdings = pd.DataFrame()
reportdate = ''
fundname = ''

legend = pd.DataFrame()

identifiercount = 0

headername = ''


for df in tables:
    #remove all rows and columns of nas
    df = df.dropna(axis = 0, how = 'all')
    df = df.dropna(axis = 1, how = 'all')
    
    if not df.empty:
        
        #get the report dates and fund names of this dataframe
        reportdate_test, fundname_test = get_report_fund(df)
        if reportdate_test != '':
            found = True
            reportdate = reportdate_test
            fundname = fundname_test
        
        
        
        for count, row in df.iterrows():
            firstrow = re.sub(r'\s', '', str(row[df.columns[0]])).encode('ascii','ignore').decode().lower()
            if '(' in firstrow and ')' in firstrow and len(firstrow) == 3:
                
                if len(holdings.loc[holdings['identifier'] == '']) > 0:
                    identifiercount = identifiercount + 1
                    found = False
                    holdings.loc[holdings['identifier'] == '', 'identifier'] = text + str(identifiercount)
                    
                add_legend = pd.DataFrame(row).transpose()
                add_legend.columns = ['symbol', 'code']
                add_legend.loc[:, 'identifier'] = text + str(identifiercount)
                
                if len(legend) == 0:
                    legend = add_legend.copy()

                else:
                    if add_legend['symbol'].values[0] not in legend.loc[legend['identifier'] ==\
                                                                    text + str(identifiercount), 'symbol'].tolist():
                            legend = pd.concat([legend, add_legend], axis = 0)
                    
                    

        cleaned_df, headername = get_holdings(df, reportdate, fundname, headername)



        if len(cleaned_df) > 0 and found:
            security = cleaned_df.copy()
            security['holdings name'] = security['holdings name'].apply(lambda x: x.encode('ascii','ignore').decode())
            if len(security) > 0:
                if holdings.empty:
                    holdings = security.copy()
                else:
                    holdings = pd.concat([holdings, security], axis = 0)



legend.index = pd.RangeIndex(len(legend.index))
holdings.index = pd.RangeIndex(len(holdings.index))


            
                



In [1]:

holdings

NameError: name 'holdings' is not defined

In [ ]:
holdings['value multiplier'] = ''
holdings['key'] = ''
holdings['textfile'] = text
holdings['CIK'] = CIK
holdings['acq name'] = ''
holdings['acq date'] = ''
holdings['acq cost'] = ''
holdings['date_filed'] = '' 
holdings['company conformed name'] = ''
try:
    date_filed = panel2.loc[panel2['file_read'] == text, 'date_filed'].unique()[0]
    conformed_name = panel2.loc[panel2['file_read'] == text, 'company conformed name'].unique()[0]
except IndexError:
    date_filed = ''
    conformed_name = ''
    
holdings.loc[:, 'date_filed'] = date_filed
holdings.loc[:, 'company conformed name'] = conformed_name

holdings.index = pd.RangeIndex(len(holdings.index))
print(holdings)

for z in weblink[CIK]:
    if text in z:
        matching_link = z

holdings['weblink'] = matching_link
legend['weblink'] = matching_link
legend['textfile'] = text

get the acquisition data

In [ ]:
holdings = get_keys(holdings)

In [ ]:
holdings

In [ ]:
holdings['identifier']

In [ ]:
legend.loc[:,'restricted'] = 'no'

legend.loc[legend['code'].str.contains("restrict") | legend['code'].str.contains("level 3 security") \
              | legend['code'].str.contains("exempt from registration"), 'restricted'] = 'yes'

restricted = legend.loc[legend['restricted'].str.contains("yes")]
print(restricted)

holdings.loc[:,'restricted'] = 'no'


In [ ]:
#get the restricted legend and restricted securities

In [ ]:
legend.loc[:,'restricted'] = 'no'

legend.loc[legend['code'].str.contains("restrict") | legend['code'].str.contains("level 3 security") \
              | legend['code'].str.contains("exempt from registration"), 'restricted'] = 'yes'

restricted = legend.loc[legend['restricted'].str.contains("yes")]
print(restricted)

holdings.loc[:,'restricted'] = 'no'

for index, row in restricted.iterrows():
    holdings.loc[(holdings['identifier'] == row['identifier']) & (holdings['textfile'] == row.textfile) \
                   & (holdings['key'].str.contains(row['symbol'])), 'restricted'] = 'yes'
    
restricted_holdings = holdings.loc[(holdings['restricted'] == 'yes')]

In [ ]:
#get the restricted holdings

In [ ]:
print(restricted_holdings)

In [ ]:
def getpanel1(CIK, text, panel2):
    
    url = r'https://www.sec.gov/Archives/edgar/data/' + CIK +  r'/' + text
    tables = pd.read_html(url,encoding="utf-8")
    
    holdings = pd.DataFrame()
    reportdate = ''
    fundname = ''

    legend = pd.DataFrame()

    identifiercount = 0

    headername = ''


    for df in tables:
        #remove all rows and columns of nas
        df = df.dropna(axis = 0, how = 'all')
        df = df.dropna(axis = 1, how = 'all')

        if not df.empty:

            #get the report dates and fund names of this dataframe
            reportdate_test, fundname_test = get_report_fund(df)
            if reportdate_test != '':
                found = True
                reportdate = reportdate_test
                fundname = fundname_test



            for count, row in df.iterrows():
                firstrow = re.sub(r'\s', '', str(row[df.columns[0]])).encode('ascii','ignore').decode().lower()
                if '(' in firstrow and ')' in firstrow and len(firstrow) == 3:

                    if len(holdings.loc[holdings['identifier'] == '']) > 0:
                        identifiercount = identifiercount + 1
                        found = False
                        holdings.loc[holdings['identifier'] == '', 'identifier'] = text + str(identifiercount)

                    add_legend = pd.DataFrame(row).transpose()
                    add_legend.columns = ['symbol', 'code']
                    add_legend.loc[:, 'identifier'] = text + str(identifiercount)

                    if len(legend) == 0:
                        legend = add_legend.copy()

                    else:
                        if add_legend['symbol'].values[0] not in legend.loc[legend['identifier'] ==\
                                                                        text + str(identifiercount), 'symbol'].tolist():
                                legend = pd.concat([legend, add_legend], axis = 0)



            cleaned_df, headername = get_holdings(df, reportdate, fundname, headername)



            if len(cleaned_df) > 0 and found:
                security = cleaned_df.copy()
                security['holdings name'] = security['holdings name'].apply(lambda x: x.encode('ascii','ignore').decode())
                if len(security) > 0:
                    if holdings.empty:
                        holdings = security.copy()
                    else:
                        holdings = pd.concat([holdings, security], axis = 0)



    legend.index = pd.RangeIndex(len(legend.index))
    holdings.index = pd.RangeIndex(len(holdings.index))
    
    
    
    holdings['value multiplier'] = ''
    holdings['key'] = ''
    holdings['textfile'] = text
    holdings['CIK'] = CIK
    holdings['acq name'] = ''
    holdings['acq date'] = ''
    holdings['acq cost'] = ''
    holdings['date_filed'] = '' 
    holdings['company conformed name'] = ''
    
    try:
        date_filed = panel2.loc[panel2['file_read'] == text, 'date_filed'].unique()[0]
        conformed_name = panel2.loc[panel2['file_read'] == text, 'company conformed name'].unique()[0]
    except IndexError:
        date_filed = ''
        conformed_name = ''

    holdings.loc[:, 'date_filed'] = date_filed
    holdings.loc[:, 'company conformed name'] = conformed_name

    holdings.index = pd.RangeIndex(len(holdings.index))

    holdings['weblink'] = url
    legend['weblink'] = url
    legend['textfile'] = text
    
    
    holdings = get_keys(holdings)
        

        
    return holdings, legend




## Run on All CIKS

In [ ]:
panel1 = pd.DataFrame()


panel1 = pd.DataFrame()
panel1_legend = pd.DataFrame()
error_panel1 = []

#CIK_LIST = ['319108','1097293','230382']

CIK_LIST = ['844779', '1398078', '774013', '1062806', '1026144', '1062805', '276463', '1097077', '1137391', '1160864', '1181249', '1159038', '353281', '834237', '97098', '216557', '765199', '814507', '790525', '110055', '1159040', '1137393', '1176194', '877151', '1169029', '1167467', '319108', '1051003', '1137440', '1167470','230382','1097293']


for CIK in CIK_LIST:
    txt_files = os.listdir(os.path.join(cur_wd, CIK, 'n-q'))
    
    for i in txt_files:
        if '.csv' not in i:
            try:
                holdings, legend = getpanel1(CIK, i, panel2)

                if len(holdings) == 0:
                    #print(i)
                    pass
                if panel1.empty:
                    panel1 = holdings.copy()
                    panel1_legend = legend.copy()
                else:
                    panel1 = pd.concat([panel1, holdings], axis = 0, ignore_index = True)
                    panel1_legend = pd.concat([panel1_legend , legend], axis = 0, ignore_index = True)
                panel1.index = pd.RangeIndex(len(panel1.index))
            except:
                error_panel1.append(r'https://www.sec.gov/Archives/edgar/data/' + CIK +  r'/' + i)

In [ ]:
error_panel1

In [ ]:
print(panel1.dropna(axis =1, how = 'all'))
panel1.to_csv(os.path.join(output_directory, fund_name + '_panel1.csv'), sep = ',')
panel1_legend.to_csv(os.path.join(output_directory, fund_name + '_panel1_legend.csv'), sep = ',')

In [ ]:
panel1_legend.loc[:,'restricted'] = 'no'

panel1_legend.loc[panel1_legend['code'].str.contains("restrict") | panel1_legend['code'].str.contains("level 3 security") \
              | panel1_legend['code'].str.contains("exempt from registration"), 'restricted'] = 'yes'

restricted = panel1_legend.loc[panel1_legend['restricted'].str.contains("yes")]

restricted.to_csv(os.path.join(output_directory, fund_name + '_panel1_restricted_legend.csv'), sep = ',')
print(restricted)

panel1.loc[:,'restricted'] = 'no'

for index, row in restricted.iterrows():
    panel1.loc[(panel1['identifier'] == row['identifier']) & (panel1['textfile'] == row.textfile) \
                   & (panel1['key'].str.contains(row['symbol'])), 'restricted'] = 'yes'
    
restricted_holdings = panel1.loc[(panel1['restricted'] == 'yes')]
restricted_holdings.to_csv(os.path.join(output_directory, fund_name + '_panel1_restricted.csv'), sep = ',')